目录

一、cat对象

    1. cat对象的属性
    2. 类别的增加、删除和修改


二、有序分类

    1. 序的建立
    2. 排序和比较

三、区间类别

    1. 利用cut和qcut进行区间构造
    2. 一般区间的构造
    练一练
    3. 区间的属性与方法

四、练习

<center><h1>第九章 分类数据</h1></center>

In [1]:
import numpy as np
import pandas as pd

## 一、cat对象
### 1. cat对象的属性


In [3]:
# 在pandas中提供了category类型，使用户能够处理分类类型的变量，将一个普通序列转换成分类变量可以使用astype方法。
df = pd.read_csv('./data/learn_pandas.csv', usecols = ['Grade', 'Name', 'Gender', 'Height', 'Weight'])
s = df.Grade.astype('category')
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Junior', 'Senior', 'Sophomore']

In [5]:
# 在一个分类类型的Series中定义了cat对象，定义了一些属性和方法来进行分类类别的操作。
s.cat

In [6]:
# 对于一个具体的分类，有两个组成部分，其一为类别的本身，它以Index类型存储
s.cat.categories

Index(['Freshman', 'Junior', 'Senior', 'Sophomore'], dtype='object')

In [7]:
# 其二为是否有序
s.cat.ordered

False

In [8]:
# 每一个序列的类别会被赋予唯一的整数编号，它们的编号取决于cat.categories中的顺序，该属性可以通过codes访问
s.cat.codes.head()

0    0
1    0
2    2
3    3
4    3
dtype: int8

### 2. 类别的增加、删除和修改



#### 【NOTE】类别不得直接修改
在第三章中曾提到，索引 ``Index`` 类型是无法用 ``index_obj[0] = item`` 来修改的，而 ``categories`` 被存储在 ``Index`` 中，因此 ``pandas`` 在 ``cat`` 属性上定义了若干方法来达到相同的目的。
#### 【END】


In [9]:
# 类别的增加可以使用`add_categories`：
s = s.cat.add_categories('Graduate') # 增加一个毕业生类别
s.cat.categories

Index(['Freshman', 'Junior', 'Senior', 'Sophomore', 'Graduate'], dtype='object')

In [10]:
# 删除某一个类别可以使用remove_categories，
# 删除大一的类别
s = s.cat.remove_categories('Freshman')
s.cat.categories

Index(['Junior', 'Senior', 'Sophomore', 'Graduate'], dtype='object')

In [11]:
# 删除某一个类别后所有原来序列中的该类会被设置为缺失
s.head()

0          NaN
1          NaN
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Junior', 'Senior', 'Sophomore', 'Graduate']

In [12]:
# 使用set_categories直接设置序列的新类别
s = s.cat.set_categories(['Sophomore','PhD']) # 新类别为大二学生和博士
s.cat.categories

Index(['Sophomore', 'PhD'], dtype='object')

In [13]:
# 原来的类别中如果存在元素不属于新类别，那么会被设置为缺失
s.head()

0          NaN
1          NaN
2          NaN
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (2, object): ['Sophomore', 'PhD']

In [14]:
# 删除未出现在序列中的类别，可以使用remove_unused_categories来实现
s = s.cat.remove_unused_categories() # 移除了未出现的博士生类别
s.cat.categories

Index(['Sophomore'], dtype='object')

In [15]:
# 通过rename_categories方法完成修改的操作，这个方法会对原序列的对应值也进行相应修改
# 把Sophomore改成中文的本科二年级学生
s = s.cat.rename_categories({'Sophomore':'本科二年级学生'})
s.head()

0        NaN
1        NaN
2        NaN
3    本科二年级学生
4    本科二年级学生
Name: Grade, dtype: category
Categories (1, object): ['本科二年级学生']

## 二、有序分类
### 1. 序的建立


In [16]:
# 有序类别和无序类别可以通过as_unordered和reorder_categories互相转化
# reorder_categories传入的参数必须是由当前序列的无序类别构成的列表，不能够增加新的类别，也不能缺少原来的类别，
# 并且必须指定参数ordered=True，否则方法无效

# 对年级高低进行相对大小的类别划分
s = df.Grade.astype('category')
s = s.cat.reorder_categories(['Freshman', 'Sophomore', 'Junior', 'Senior'],ordered=True)
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman' < 'Sophomore' < 'Junior' < 'Senior']

In [17]:
# 恢复无序状态
s.cat.as_unordered().head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Sophomore', 'Junior', 'Senior']

In [18]:
# 如果不想指定ordered=True参数，那么可以先用s.cat.as_ordered()转化为有序类别，
s = df.Grade.astype('category')
s = s.cat.as_ordered()
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman' < 'Junior' < 'Senior' < 'Sophomore']

In [20]:
# 再利用reorder_categories进行具体的相对大小调整
s = s.cat.reorder_categories(['Freshman', 'Sophomore', 'Junior', 'Senior'])
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman' < 'Sophomore' < 'Junior' < 'Senior']


### 2. 排序和比较


In [21]:
# 分类变量的排序：只需把列的类型修改为category后，再赋予相应的大小关系，就能正常地使用sort_index和sort_values。

# 对年级进行排序
df.Grade = df.Grade.astype('category')
df.Grade = df.Grade.cat.reorder_categories(['Freshman', 'Sophomore', 'Junior', 'Senior'],ordered=True)
df.sort_values('Grade').head() # 值排序

,Grade,Name,Gender,Height,Weight
0,Freshman,Gaopeng Yang,Female,158.9,46.0
105,Freshman,Qiang Shi,Female,164.5,52.0
96,Freshman,Changmei Feng,Female,163.8,56.0
88,Freshman,Xiaopeng Han,Female,164.1,53.0
81,Freshman,Yanli Zhang,Female,165.1,52.0


In [22]:
df.set_index('Grade').sort_index().head() # 索引排序

,Name,Gender,Height,Weight
Grade,,,,
Freshman,Gaopeng Yang,Female,158.9,46.0
Freshman,Qiang Shi,Female,164.5,52.0
Freshman,Changmei Feng,Female,163.8,56.0
Freshman,Xiaopeng Han,Female,164.1,53.0
Freshman,Yanli Zhang,Female,165.1,52.0


In [23]:
# 分类变量的比较操作分为两类
# 第一种是==或!=关系的比较，比较的对象可以是标量或者同长度的Series（或list）
res1 = df.Grade == 'Sophomore'
res1.head()

0    False
1    False
2    False
3     True
4     True
Name: Grade, dtype: bool

In [24]:
res2 = df.Grade == ['PhD']*df.shape[0]
res2.head()

0    False
1    False
2    False
3    False
4    False
Name: Grade, dtype: bool

In [25]:
# 第二种是>,>=,<,<=四类大小关系的比较，比较的对象和第一种类似，
# 但是所有参与比较的元素必须属于原序列的categories，同时要和原序列具有相同的索引。
res3 = df.Grade <= 'Sophomore'
res3.head()

0     True
1     True
2    False
3     True
4     True
Name: Grade, dtype: bool

In [26]:
res4 = df.Grade <= df.Grade.sample(frac=1).reset_index(drop=True) # 打乱后比较
res4.head()

0     True
1     True
2    False
3     True
4    False
Name: Grade, dtype: bool

## 三、区间类别

### 1. 利用cut和qcut进行区间构造

区间是一种特殊的类别，在实际数据分析中，区间序列往往是通过`cut`和`qcut`方法进行构造的，这两个函数能够把原序列的数值特征进行装箱，即用区间位置来代替原来的具体数值。


In [27]:
# cut最重要的参数是bins
# 如果传入整数n，则代表把整个传入数组的按照最大和最小值等间距地分为n段。

# 由于区间默认是左开右闭，需要在调整时把最小值包含进去，
# 在pandas中的解决方案是在值最小的区间左端点再减去0.001*(max-min)

# 如果对序列[1,2]划分为2个箱子时，第一个箱子的范围(0.999,1.5]，第二个箱子的范围是(1.5,2]。

s = pd.Series([1,2])
pd.cut(s, bins=2)

0    (0.999, 1.5]
1      (1.5, 2.0]
dtype: category
Categories (2, interval[float64]): [(0.999, 1.5] < (1.5, 2.0]]

In [28]:
# 需要指定左闭右开时，需要把right参数设置为False，相应的区间调整方法是在值最大的区间右端点再加上0.001*(max-min)。
pd.cut(s, bins=2, right=False)

0      [1.0, 1.5)
1    [1.5, 2.001)
dtype: category
Categories (2, interval[float64]): [[1.0, 1.5) < [1.5, 2.001)]

In [29]:
# bins的另一个常见用法是指定区间分割点的列表（使用np.infty可以表示无穷大）：
pd.cut(s, bins=[-np.infty, 1.2, 1.8, 2.2, np.infty])

0    (-inf, 1.2]
1     (1.8, 2.2]
dtype: category
Categories (4, interval[float64]): [(-inf, 1.2] < (1.2, 1.8] < (1.8, 2.2] < (2.2, inf]]

In [34]:
# 另外两个常用参数为labels和retbins
# labels代表了区间的名字
# retbins代表了是否返回分割点（默认不返回）
s = pd.Series([1,2])
res = pd.cut(s, bins=2, labels=['small', 'big'], retbins=True)
res[0]

0    small
1      big
dtype: category
Categories (2, object): ['small' < 'big']

In [35]:
res[1] # 该元素为返回的分割点

array([0.999, 1.5  , 2.   ])

In [36]:
s = pd.Series([1,2])
res = pd.cut(s, bins=2, labels=['small', 'big'], retbins=False)
res[0]

'small'

In [37]:
res[1]

'big'

In [38]:
# qcut中的q是指quantile。还可以传入浮点列表指代相应的分位数分割点。
# 这里的q为整数n时，指按照n等分位数把数据分箱，
s = df.Weight
pd.qcut(s, q=3).head()

0    (33.999, 48.0]
1      (55.0, 89.0]
2      (55.0, 89.0]
3    (33.999, 48.0]
4      (55.0, 89.0]
Name: Weight, dtype: category
Categories (3, interval[float64]): [(33.999, 48.0] < (48.0, 55.0] < (55.0, 89.0]]

In [39]:
# 还可以传入浮点列表指代相应的分位数分割点
pd.qcut(s, q=[0,0.2,0.8,1]).head()

0      (44.0, 69.4]
1      (69.4, 89.0]
2      (69.4, 89.0]
3    (33.999, 44.0]
4      (69.4, 89.0]
Name: Weight, dtype: category
Categories (3, interval[float64]): [(33.999, 44.0] < (44.0, 69.4] < (69.4, 89.0]]

### 2. 一般区间的构造


In [40]:
# 对于某一个具体的区间而言，其具备三个要素，即左端点、右端点和端点的开闭状态，
# 其中开闭状态可以指定right, left, both, neither中的一类
my_interval = pd.Interval(0, 1, 'right')
my_interval

Interval(0, 1, closed='right')

In [43]:
# 其属性包含了mid中点, length长度, right右端点, left左端点, closed开闭状态

# 使用in可以判断元素是否属于区间
0.5 in my_interval

True

In [44]:
# 使用overlaps可以判断两个区间是否有交集
my_interval_2 = pd.Interval(0.5, 1.5, 'left')
my_interval.overlaps(my_interval_2)

True

In [45]:
# pd.IntervalIndex对象有四类方法生成，分别是from_breaks, from_arrays, from_tuples, interval_range

# from_breaks的功能类似于cut或qcut函数
# cut或qcut函数通过计算得到的分割点
# from_breaks直接传入自定义的分割点
pd.IntervalIndex.from_breaks([1,3,6,10], closed='both')

IntervalIndex([[1, 3], [3, 6], [6, 10]],
              closed='both',
              dtype='interval[int64]')

In [46]:
# from_arrays是分别传入左端点和右端点的列表，适用于有交集并且知道起点和终点的情况
pd.IntervalIndex.from_arrays(left = [1,3,6,10], right = [5,4,9,11], closed = 'neither')

IntervalIndex([(1, 5), (3, 4), (6, 9), (10, 11)],
              closed='neither',
              dtype='interval[int64]')

In [47]:
# from_tuples传入的是起点和终点元组构成的列表
pd.IntervalIndex.from_tuples([(1,5),(3,4),(6,9),(10,11)], closed='neither')

IntervalIndex([(1, 5), (3, 4), (6, 9), (10, 11)],
              closed='neither',
              dtype='interval[int64]')

In [48]:
# 一个等差的区间序列由 起点start、 终点end、 区间个数periods 和 区间长度freq决定
# 其中三个量确定的情况下，剩下一个量就确定了
pd.interval_range(start=1,end=5,periods=8)

IntervalIndex([(1.0, 1.5], (1.5, 2.0], (2.0, 2.5], (2.5, 3.0], (3.0, 3.5], (3.5, 4.0], (4.0, 4.5], (4.5, 5.0]],
              closed='right',
              dtype='interval[float64]')

In [49]:
pd.interval_range(end=5,periods=8,freq=0.5)

IntervalIndex([(1.0, 1.5], (1.5, 2.0], (2.0, 2.5], (2.5, 3.0], (3.0, 3.5], (3.5, 4.0], (4.0, 4.5], (4.5, 5.0]],
              closed='right',
              dtype='interval[float64]')

#### 【练一练】
无论是`interval_range`还是下一章时间序列中的`date_range`都是给定了等差序列中四要素中的三个，从而确定整个序列。请回顾等差数列中的首项、末项、项数和公差的联系，写出`interval_range`中四个参数之间的恒等关系。


In [ ]:
终点end = 起点start +  区间个数periods * 区间长度freq

In [50]:
my_interval

Interval(0, 1, closed='right')

In [51]:
my_interval_2

Interval(0.5, 1.5, closed='left')

In [52]:
# 如果直接使用pd.IntervalIndex([...], closed=...)，把Interval类型的列表组成传入其中转为区间索引
# 那么所有的区间会被强制转为指定的closed类型，
# 因为pd.IntervalIndex只允许存放同一种开闭区间的Interval对象
pd.IntervalIndex([my_interval, my_interval_2], closed='left')

IntervalIndex([[0.0, 1.0), [0.5, 1.5)],
              closed='left',
              dtype='interval[float64]')

### 3. 区间的属性与方法


In [53]:
# IntervalIndex上也定义了一些有用的属性和方法。同时，如果想要具体利用cut或者qcut的结果进行分析，那么需要先将其转为该种索引类型
id_interval = pd.IntervalIndex(pd.cut(s, 3))
id_interval[:3]

IntervalIndex([(33.945, 52.333], (52.333, 70.667], (70.667, 89.0]],
              closed='right',
              name='Weight',
              dtype='interval[float64]')

In [54]:
# 与单个Interval类型相似，IntervalIndex有若干常用属性：left, right, mid, length

id_demo = id_interval[:5] # 选出前5个展示
id_demo

IntervalIndex([(33.945, 52.333], (52.333, 70.667], (70.667, 89.0], (33.945, 52.333], (70.667, 89.0]],
              closed='right',
              name='Weight',
              dtype='interval[float64]')

In [55]:
# left左端点
id_demo.left

Float64Index([33.945, 52.333, 70.667, 33.945, 70.667], dtype='float64')

In [56]:
# right右端点
id_demo.right

Float64Index([52.333, 70.667, 89.0, 52.333, 89.0], dtype='float64')

In [57]:
# 两端点均值
id_demo.mid

Float64Index([43.138999999999996, 61.5, 79.8335, 43.138999999999996, 79.8335], dtype='float64')

In [58]:
# 区间长度
id_demo.length

Float64Index([18.387999999999998, 18.334000000000003, 18.333,
              18.387999999999998, 18.333],
             dtype='float64')

In [59]:
# IntervalIndex还有两个常用方法，包括contains和overlaps

# contains指逐个判断每个区间是否包含某元素
id_demo.contains(4)

array([False, False, False, False, False])

In [60]:
# overlaps指是否和一个pd.Interval对象有交集
id_demo.overlaps(pd.Interval(40,60))

array([ True,  True, False,  True, False])

## 四、练习
### Ex1：统计未出现的类别

在第五章中介绍了`crosstab`函数，在默认参数下它能够对两个列的组合出现的频数进行统计汇总：

In [84]:
df = pd.DataFrame({'A':['a','b','c','a'], 'B':['cat','cat','dog','cat']})
df

,A,B
0,a,cat
1,b,cat
2,c,dog
3,a,cat


In [85]:
pd.crosstab(df.A, df.B)

B,cat,dog
A,,
a,2,0
b,1,0
c,0,1


但事实上有些列存储的是分类变量，列中并不一定包含所有的类别，此时如果想要对这些未出现的类别在`crosstab`结果中也进行汇总，则可以指定`dropna`参数为`False`：

In [62]:
df.B = df.B.astype('category').cat.add_categories('sheep')
pd.crosstab(df.A, df.B, dropna=False)

B,cat,dog,sheep
A,,,
a,2,0,0
b,1,0,0
c,0,1,0


In [63]:
# 请实现一个带有dropna参数的my_crosstab函数来完成上面的功能。

def my_crosstab(s1, s2, dropna=True):
    idx1 = (s1.cat.categories if s1.dtype.name == 'category' and
                             not dropna else s1.unique())
    idx2 = (s2.cat.categories if s2.dtype.name == 'category' and
                             not dropna else s2.unique())
    res = pd.DataFrame(np.zeros((idx1.shape[0], idx2.shape[0])),
                    index=idx1, columns=idx2)
    for i, j in zip(s1, s2):
        res.at[i, j] += 1
    res = res.rename_axis(index=s1.name, columns=s2.name).astype('int')
    return res


df = pd.DataFrame({'A':['a','b','c','a'],
                   'B':['cat','cat','dog','cat']})


df.B = df.B.astype('category').cat.add_categories('sheep')

my_crosstab(df.A, df.B)

B,cat,dog
A,,
a,2,0
b,1,0
c,0,1


In [64]:
my_crosstab(df.A, df.B, dropna=False)

B,cat,dog,sheep
A,,,
a,2,0,0
b,1,0,0
c,0,1,0




### Ex2：钻石数据集

现有一份关于钻石的数据集，其中`carat, cut, clarity, price`分别表示克拉重量、切割质量、纯净度和价格，样例如下：

In [88]:
df = pd.read_csv('./data/diamonds.csv') 
df.head(3)

,carat,cut,clarity,price
0,0.23,Ideal,SI2,326
1,0.21,Premium,SI1,326
2,0.23,Good,VS1,327


In [89]:
# 1. 分别对`df.cut`在`object`类型和`category`类型下使用`nunique`函数，并比较它们的性能。
df = pd.read_csv('data/diamonds.csv')

s_obj, s_cat = df.cut, df.cut.astype('category')

In [90]:
%timeit -n 30 s_obj.nunique()

2.85 ms ± 445 µs per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [91]:
%timeit -n 30 s_cat.nunique()

757 µs ± 272 µs per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [92]:
# 2. 钻石的切割质量可以分为五个等级，由次到好分别是`Fair, Good, Very Good, Premium, Ideal`，
# 纯净度有八个等级，由次到好分别是`I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF`，
# 请对切割质量按照**由好到次**的顺序排序，相同切割质量的钻石，按照纯净度进行**由次到好**的排序。

df.cut = df.cut.astype('category').cat.reorder_categories([
        'Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],ordered=True)


df.clarity = df.clarity.astype('category').cat.reorder_categories([
        'I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF'],ordered=True)


res = df.sort_values(['cut', 'clarity'], ascending=[False, True])

res.head(3)

,carat,cut,clarity,price
315,0.96,Ideal,I1,2801
535,0.96,Ideal,I1,2826
551,0.97,Ideal,I1,2830


In [93]:
res.tail(3)

,carat,cut,clarity,price
47407,0.52,Fair,IF,1849
49683,0.52,Fair,IF,2144
50126,0.47,Fair,IF,2211


In [94]:
# 3. 分别采用两种不同的方法，把`cut, clarity`这两列按照**由好到次**的顺序，映射到从0到n-1的整数，其中n表示类别的个数。
df.cut = df.cut.cat.reorder_categories(
        df.cut.cat.categories[::-1])


df.clarity = df.clarity.cat.reorder_categories(
            df.clarity.cat.categories[::-1])


df.cut = df.cut.cat.codes # 方法一：利用cat.codes




df.head(3)

,carat,cut,clarity,price
0,0.23,0,SI2,326
1,0.21,1,SI1,326
2,0.23,3,VS1,327


In [95]:
clarity_cat = df.clarity.cat.categories

df.clarity = df.clarity.replace(dict(zip(
            clarity_cat, np.arange(
                len(clarity_cat))))) # 方法二：使用replace映射


df.head(3)

,carat,cut,clarity,price
0,0.23,0,6,326
1,0.21,1,5,326
2,0.23,3,3,327


In [96]:
# 4. 对每克拉的价格分别按照分位数（q=\[0.2, 0.4, 0.6, 0.8\]）与\[1000, 3500, 5500, 18000\]割点进行分箱
# 得到五个类别`Very Low, Low, Mid, High, Very High`，并把按这两种分箱方法得到的`category`序列依次添加到原表中。
q = [0, 0.2, 0.4, 0.6, 0.8, 1]

point = [-np.infty, 1000, 3500, 5500, 18000, np.infty]

avg = df.price / df.carat

df['avg_cut'] = pd.cut(avg, bins=point, labels=[
                'Very Low', 'Low', 'Mid', 'High', 'Very High'])


df['avg_qcut'] = pd.qcut(avg, q=q, labels=[
                'Very Low', 'Low', 'Mid', 'High', 'Very High'])


df.head()

,carat,cut,clarity,price,avg_cut,avg_qcut
0,0.23,0,6,326,Low,Very Low
1,0.21,1,5,326,Low,Very Low
2,0.23,3,3,327,Low,Very Low
3,0.29,1,4,334,Low,Very Low
4,0.31,3,6,335,Low,Very Low


In [97]:
# 5. 第4问中按照整数分箱得到的序列中，是否出现了所有的类别？如果存在没有出现的类别请把该类别删除。
df.avg_cut.unique()

['Low', 'Mid', 'High']
Categories (3, object): ['Low' < 'Mid' < 'High']

In [98]:
df.avg_cut.cat.categories

Index(['Very Low', 'Low', 'Mid', 'High', 'Very High'], dtype='object')

In [99]:
df.avg_cut = df.avg_cut.cat.remove_categories([
            'Very Low', 'Very High'])


df.avg_cut.head(3)

0    Low
1    Low
2    Low
Name: avg_cut, dtype: category
Categories (3, object): ['Low' < 'Mid' < 'High']

In [80]:
# 6. 对第4问中按照分位数分箱得到的序列，求每个样本对应所在区间的左右端点值和长度。

In [101]:
interval_avg = pd.IntervalIndex(pd.qcut(avg, q=q))

interval_avg.right.to_series().reset_index(drop=True).head(3)

0    2295.0
1    2295.0
2    2295.0
dtype: float64

In [102]:
interval_avg.left.to_series().reset_index(drop=True).head(3)

0    1051.162
1    1051.162
2    1051.162
dtype: float64

In [103]:
interval_avg.length.to_series().reset_index(drop=True).head(3)

0    1243.838
1    1243.838
2    1243.838
dtype: float64